In [75]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import textblob as tb
import xgboost
import re
from textblob import TextBlob
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [76]:
users=pd.read_csv(r'C:\Users\nigal\Downloads\Twitter_Users.csv',encoding='latin1')
tweets=pd.read_csv(r'C:\Users\nigal\Downloads\Twitter_Data.csv')
df=pd.merge(tweets,users,on='Twitter_User_Name',how='inner')

In [78]:
count_bots=len(df[df['Label']==1])
count_humans=len(df[df['Label']==0])
print(count_bots," ",count_humans)
print('\n')

tweet_counts=df.groupby('Twitter_User_Name').size().reset_index(name='tweet_count')
user_tweets=pd.merge(tweets,tweet_counts,on='Twitter_User_Name',how='inner')
print(user_tweets.groupby('Label')['tweet_count'].mean())

follower_counts=df.groupby('Label')['Followers'].sum()

144224   135467


Label
0    3178.666221
1    3540.641308
Name: tweet_count, dtype: float64


In [79]:
followers_bots_mean=follower_counts[1]/count_bots
followers_humans_mean=follower_counts[0]/count_humans
print(followers_bots_mean,"\n",followers_humans_mean)

2876888.7670013313 
 26185.005499494342


In [80]:
def tweet_clean(text):
    text=text.lower()
    text=re.sub(r'https\S+','',text)
    text=re.sub(r'@\w+','',text)
    text=re.sub(r'#\w+','',text)
    text=re.sub(r'[^a-z\s]','',text)
    return text

In [81]:
df['Clean_tweet']=df['Tweet_text'].apply(tweet_clean)
df['Tweet_length']=df['Clean_tweet'].str.split().str.len()

In [82]:

def count_mentions(text):
    if not isinstance(text,str):
        return 0
    return len(re.findall(r'@\w+',text))
def count_hashtags(text):
    if not isinstance(text,str):
        return 0
    return len(re.findall(r'#\w+',text))
def count_urls(text):
    if not isinstance(text,str):
        return 0
    return len(re.findall(r'http[s]?://\S+|www\.\S+',text))
def count_retweet(text):
    if text.startswith('RT @'):
        return 1
    return 0
    

df['Mention_count']=df['Tweet_text'].apply(count_mentions)
df['Hashtag_count']=df['Tweet_text'].apply(count_hashtags)
df['URL_count']=df['Tweet_text'].apply(count_urls)
df['Retweet_count']=df['Tweet_text'].apply(count_retweet)
df['Real_Location']=df['Real_Location'].fillna(0)
df['Tweet_created_at']=(pd.to_datetime(df['Tweet_created_at'])).astype('int64')/1e9
df['sentiment']=df['Clean_tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [67]:
X=df[['Tweet_created_at','Following','Followers','Verified','Tweet_length','Mention_count',
      'Hashtag_count','URL_count','Retweet_count','Real_Location','sentiment']]
y=df['Label']


In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
kf=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

roc_auc_scores=[]
fold=1

for train_index,test_index in kf.split(X,y):
    X_train,X_test=X.iloc[train_index],X.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]

    model=XGBClassifier(n_estimators=60,max_depth=2,learning_rate=0.1)
    model.fit(X_train,y_train)
    
    y_pred=model.predict(X_test)
    y_prob=model.predict_proba(X_test)[:,1]
    
    print(f"\n=== Fold {fold} ===")
    print(classification_report(y_test,y_pred))
    roc_auc=roc_auc_score(y_test, y_prob)
    print(f"ROC-AUC: {roc_auc:.4f}")
    
    roc_auc_scores.append(roc_auc)
    fold+=1
    train_prob=model.predict_proba(X_train)[:,1]
    train_auc=roc_auc_score(y_train, train_prob)
    
    val_prob= y_prob=model.predict_proba(X_test)[:,1]
    val_auc=roc_auc_score(y_test, val_prob)
    
    print("Train ROC-AUC:",train_auc)
    print("Validation ROC-AUC:",val_auc)
print(f"\nAverage ROC-AUC: {np.mean(roc_auc_scores):.4f}")



=== Fold 1 ===
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     27094
           1       0.96      0.97      0.97     28845

    accuracy                           0.96     55939
   macro avg       0.96      0.96      0.96     55939
weighted avg       0.96      0.96      0.96     55939

ROC-AUC: 0.9934
Train ROC-AUC: 0.9936368122371467
Validation ROC-AUC: 0.9934296495897138

=== Fold 2 ===
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     27094
           1       0.96      0.95      0.96     28844

    accuracy                           0.96     55938
   macro avg       0.96      0.96      0.96     55938
weighted avg       0.96      0.96      0.96     55938

ROC-AUC: 0.9952
Train ROC-AUC: 0.9950650733930158
Validation ROC-AUC: 0.9952495404295519

=== Fold 3 ===
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     27093
           